In [5]:
import json
import time
import re
from api_request.gpt35_turbo_completion import gpt35_turbo_completion


ontology = """Your task is to find the changed domain-slots based on the context and the dialogue between user and system, and find the corresponding value.
The following lists are domain-slots and their possible values.
you don't have to find other changed domain-slots if they are not in the list.

hotel-name: a and b guest house, ashley hotel, el shaddia guest house, etc.
hotel-pricerange: dontcare, cheap, moderate, expensive
hotel-type: hotel, guest house
hotel-parking: dontcare, yes, no
hotel-book stay: 1, 2, 3, etc.
hotel-book day: monday, tuesday, etc.
hotel-book people: 1, 2, 3, etc.
hotel-area: dontcare, centre, east, north, south, west
hotel-stars: dontcare, 0, 1, 2, 3, 4, 5
hotel-internet: dontcare, yes, no

train-destination: london kings cross, cambridge, peterborough, etc.
train-departure: cambridge, stansted airport, etc.
train-day: monday, saturday, etc.
train-book people: 1, 2, 3, etc.
train-depart_time: 20:24, 12:06, etc.
train-arrive_by_time: 05:51, 20:52, etc.

attraction-name: abbey pool and astroturf pitch, adc theatre, all saints church, castle galleries, etc.
attraction-area: dontcare, centre, east, north, south, west
attraction-type: architecture, boat, church, cinema, college, concert hall, entertainment, hotspot, multiple sports, museum, nightclub, park, special, swimming pool, theatre

restaurant-name: pizza hut city centre, the missing sock, golden wok, cambridge chop house, darrys cookhouse and wine shop, etc.
restaurant-food_type: italian, international, chinese, dontcare, modern european, etc.
restaurant-pricerange: dontcare, cheap, moderate, expensive
restaurant-area: centre, east, north, south, west
restaurant-book time: 13:30, 17:11, etc.
restaurant-book day: wednesday, friday, etc.
restaurant-book people: 1, 2, 3, etc.

taxi-destination: copper kettle, magdalene college, lovell lodge
taxi-departure: royal spice, university arms hotel, da vinci pizzeria
taxi-depart_time: 14:45, 11:15, etc.
taxi-arrive_by_time: 15:30, 12:45, etc.
"""

ontology = """hotel-name: a and b guest house, ashley hotel, el shaddia guest house, etc.
hotel-pricerange: dontcare, cheap, moderate, expensive
hotel-type: hotel, guest house
hotel-parking: dontcare, yes, no
hotel-book stay: 1, 2, 3, etc.
hotel-book day: monday, tuesday, etc.
hotel-book people: 1, 2, 3, etc.
hotel-area: dontcare, centre, east, north, south, west
hotel-stars: dontcare, 0, 1, 2, 3, 4, 5
hotel-internet: dontcare, yes, no

train-destination: london kings cross, cambridge, peterborough, etc.
train-departure: cambridge, stansted airport, etc.
train-day: monday, saturday, etc.
train-book people: 1, 2, 3, etc.
train-depart_time: 20:24, 12:06, etc.
train-arrive_by_time: 05:51, 20:52, etc.

attraction-name: abbey pool and astroturf pitch, adc theatre, all saints church, castle galleries, etc.
attraction-area: dontcare, centre, east, north, south, west
attraction-type: architecture, boat, church, cinema, college, concert hall, entertainment, hotspot, multiple sports, museum, nightclub, park, special, swimming pool, theatre

restaurant-name: pizza hut city centre, the missing sock, golden wok, cambridge chop house, darrys cookhouse and wine shop, etc.
restaurant-food_type: italian, international, chinese, dontcare, modern european, etc.
restaurant-pricerange: dontcare, cheap, moderate, expensive
restaurant-area: centre, east, north, south, west
restaurant-book time: 13:30, 17:11, etc.
restaurant-book day: wednesday, friday, etc.
restaurant-book people: 1, 2, 3, etc.

taxi-destination: copper kettle, magdalene college, lovell lodge
taxi-departure: royal spice, university arms hotel, da vinci pizzeria
taxi-depart_time: 14:45, 11:15, etc.
taxi-arrive_by_time: 15:30, 12:45, etc."""

slot_description_prompt = """Your task is to find the changed domain-slots based on the context and the dialogue between user and system.
The following lists are domain-slots and their descriptions.

domain-slots : description
hotel-name : name of the hotel
hotel-pricerange : price budget of the hotel
hotel-type : what is the type of the hotel
hotel-parking : parking facility at the hotel 
hotel-book stay : length of stay at the hotel
hotel-book day : day of the hotel booking
hotel-book people : number of people for the hotel booking
hotel-area : area or place of the hotel
hotel-stars : star rating of the hotel
hotel-internet : internet option at the hotel
train-destination : destination of the train
train-departure : departure location of the train
train-day : day of the train
train-book people : number of people booking for train
train-leaveat : leaving time for the train
train-arriveby : arrival time of the train
attraction-name : name of the attraction
attraction-area : area or place of the attraction
attraction-type : type of the attraction
restaurant-name : name of the restaurant
restaurant-food : food type for the restaurant 
restaurant-pricerange : price budget for the restaurant 
restaurant-area : area or place of the restaurant 
restaurant-book time : time of the restaurant booking 
restaurant-book day : day of the restaurant booking 
restaurant-book people : number of people booking the restaurant 
taxi-destination : destination of taxi 
taxi-departure : departure location of taxi
taxi-leaveat : leaving time of taxi 
taxi-arriveby : arrival time of taxi
"""

def parse_slot_with_bracket_matching(pred):

    # find all values where they are in the brackets

    # fix for no states
    if pred == "":
        return {}

    pred_slots = []

    # slot_value = pred.split(",")
    slots = re.findall(r'\((.*?)\)', pred)

    for i in slots:
        i = i.replace("(","").replace(")","")
        # for multiple slots in one bracket
        pred_slots.append(i)
    return pred_slots

In [52]:
import json
import time
from api_request.gpt35_turbo_completion import gpt35_turbo_completion

with open('data/mw24_100p_test.json', 'r') as f:
    test_data = json.load(f)
    
for idx, example in enumerate(test_data[:50]):
    print(f"\n********* {idx} ************\n")
    prompt = ""
    
    # remove multiple choice in last slot values
    last_slot_values = {s: v.split(
        '|')[0] for s, v in example['last_slot_values'].items()}

    if last_slot_values:
        prompt += f"# previous dialogue start\n"
        prompt += f"{', '.join({f'({slot} = {value})' for slot, value in last_slot_values.items()})}\n"
        prompt += f"# previous dialogue end\n\n"

    prompt += f"# current dialogue start\n"
    if example['dialog']['sys'][-1]:
        prompt += f"[system] {example['dialog']['sys'][-1]}\n"
    prompt += f"[user] {example['dialog']['usr'][-1]}\n"
    prompt += f"# current dialogue end\n\n"
    
    if example['turn_slot_values']:
        prompt += f"answer: {', '.join({f'({slot} = {value})' for slot, value in example['turn_slot_values'].items()})}\n"
        prompt += f"Based on the answer, "
    
    prompt += f"Summarize the current dialogue in a more explanatory manner while preserving the original order of it."
    
    print(prompt)
    print("\n")
    
    summary_result = gpt35_turbo_completion(prompt)
    print(summary_result.lower())
    
    print("\n\n\n")
    
    time.sleep(5)


********* 0 ************

# current dialogue start
[user] i would like a taxi from saint john s college to pizza hut fen ditton .
# current dialogue end

answer: (taxi-destination = pizza hut fenditton), (taxi-departure = saint johns college)
Based on the answer, Summarize the current dialogue in a more explanatory manner while preserving the original order of it.


the user is requesting a taxi to take them from saint john's college to pizza hut in fen ditton.





********* 1 ************

# previous dialogue start
(taxi-destination = pizza hut fenditton), (taxi-departure = saint johns college)
# previous dialogue end

# current dialogue start
[system] what time do you want to leave and what time do you want to arrive by ?
[user] i want to leave after 17:15 .
# current dialogue end

answer: (taxi-leaveat = 17:15)
Based on the answer, Summarize the current dialogue in a more explanatory manner while preserving the original order of it.


the system asks the user what time they want t

the system confirms that saffron brasserie serves indian food, which was previously established in the previous dialogue. the user expresses gratitude for the information and indicates that they do not require any further details at the moment.





********* 13 ************

# previous dialogue start
(restaurant-food = indian), (attraction-name = nusha), (restaurant-name = saffron brasserie), (restaurant-area = centre), (restaurant-pricerange = expensive)
# previous dialogue end

# current dialogue start
[system] i hope i have been of help
[user] you have . thank you . goodbye .
# current dialogue end

Summarize the current dialogue in a more explanatory manner while preserving the original order of it.


the system hopes that it has been helpful to the user. the user expresses gratitude and bids farewell.





********* 14 ************

# current dialogue start
[user] i want to find a moderate -ly priced restaurant .
# current dialogue end

answer: (restaurant-pricerange = moderate)


the system informs the user that the entrance fee for the attraction, all saints church, is free. the system then asks if there is anything else it can do for the user. the user declines and thanks the system.





********* 25 ************

# current dialogue start
[user] i am looking for a cheap restaurant in the center of the city .
# current dialogue end

answer: (restaurant-pricerange = cheap), (restaurant-area = centre)
Based on the answer, Summarize the current dialogue in a more explanatory manner while preserving the original order of it.


the user is searching for a restaurant that is affordable and located in the center of the city.





********* 26 ************

# previous dialogue start
(restaurant-pricerange = cheap), (restaurant-area = centre)
# previous dialogue end

# current dialogue start
[system] do you have any specific type of food you would like ?
[user] no , i am not picky as long as the price -s are low .
# current dialogue end

answer: (restaurant-food = don

the system informs the user that their taxi has been booked for their trip from dojo noodle bar to all saints church. the car is a blue bmw and the contact number is provided. the system asks if there is anything else it can help with, to which the user responds that there is not.





********* 35 ************

# previous dialogue start
(taxi-departure = dojo noodle bar), (attraction-type = architecture), (taxi-leaveat = 18:30), (restaurant-book time = 17:30), (attraction-area = centre), (attraction-name = all saints church), (restaurant-book day = thursday), (restaurant-pricerange = cheap), (restaurant-food = dontcare), (restaurant-area = centre), (restaurant-name = dojo noodle bar), (restaurant-book people = 8), (taxi-destination = all saints church)
# previous dialogue end

# current dialogue start
[system] awesome , enjoy your trip !
[user] thanks ! i will definitely try to .
# current dialogue end

Summarize the current dialogue in a more explanatory manner while preserving the o

the system informs the user that there are 5 trains available for booking. the system suggests a train that leaves at 7:40 and arrives at 10:23, and asks the user if they would like to book it. the user asks for the train id.





********* 46 ************

# previous dialogue start
(train-arriveby = 10:30), (train-destination = cambridge), (train-departure = birmingham new street), (train-day = saturday)
# previous dialogue end

# current dialogue start
[system] the train id is tr8259 .
[user] yes , that train sounds good . please book it for me . could you also find me a hotel with a moderate price that offers internet ?
# current dialogue end

answer: (hotel-internet = yes), (hotel-pricerange = moderate)
Based on the answer, Summarize the current dialogue in a more explanatory manner while preserving the original order of it.


the user confirms the suggested train and requests the system to book it. additionally, the user asks for a hotel with internet and a moderate price range.



RateLimitError: That model is currently overloaded with other requests. You can retry your request, or contact us through our help center at help.openai.com if the error persists. (Please include the request ID 1d52ba3d54216aa0858208531d487096 in your message.)